In [ ]:
!#pip install ultralytics
#!pip install scikit-learn

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [2]:
SEED = 42
BATCH_SIZE = 8
MODEL = "v2"

In [ ]:
if os.path.exists("data/yolo"):
    shutil.rmtree("data/yolo")

if not os.path.exists("data/yolo/train"):
    os.makedirs("data/yolo/train")
    
if not os.path.exists("data/yolo/valid"):
    os.makedirs("data/yolo/valid")
    
if not os.path.exists("data/yolo/test"):
    os.makedirs("data/yolo/test")    
    
if not os.path.exists("results"):
    os.makedirs("results")

In [ ]:
def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)        
        image_height, image_width, _ = source_image.shape
        
        target_image_path = f"data/yolo/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)
        
        if type == "test":
            continue
        
        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[5], line[7])), float(min(line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3])), float(max(line[2], line[4]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")
            
        target_label_txt = f"data/yolo/{type}/{os.path.basename(txt_path)}"      
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [ ]:
image_paths = sorted(glob("data/train/*.png"))
txt_paths = sorted(glob("data/train/*.txt"))

train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.1, random_state=SEED)

make_yolo_dataset(train_images_paths, train_txt_paths, "train")
make_yolo_dataset(valid_images_paths, valid_txt_paths, "valid")
make_yolo_dataset(sorted(glob("data/test/*.png")), None, "test")

In [ ]:
with open("data/classes.txt", "r") as reader:
    lines = reader.readlines()
    classes = [line.strip().split(",")[1] for line in lines]

yaml_data = {
              "names": classes,
              "nc": len(classes),
              "path": "data/yolo/",
              "train": "train",
              "val": "valid",
              "test": "test"
            }

with open("data/yolo/custom.yaml", "w") as writer:
    yaml.dump(yaml_data, writer)

In [9]:
#model = YOLO(f"{MODEL}/train/weights/last.pt")
model = YOLO("yolov8x.pt")
results = model.train(
    data="data\yolo\custom.yaml",
    imgsz=(640, 640),
    epochs=100,
    batch=BATCH_SIZE,
    patience=5,
    workers=16,
    device=0,
    exist_ok=True,    
    project=f"{MODEL}",
    name="train",
    seed=SEED,
    #pretrained=False,
    #resume=False,
    optimizer="AdamW",
    lr0=1e-3,
    augment=True,
    val=True,
    cache=True
    )

New https://pypi.org/project/ultralytics/8.0.112 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.109  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=data\yolo\custom.yaml, epochs=100, patience=5, batch=8, imgsz=(640, 640), save=True, save_period=-1, cache=True, device=0, workers=16, project=v2, name=train, exist_ok=True, pretrained=False, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False

In [10]:
def get_test_image_paths(test_image_paths):    
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [11]:
model = YOLO("v2/train/weights/best.pt")
test_image_paths = glob("data/yolo/test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(512, 512), iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

 99%|█████████▉| 421/425 [06:16<00:02,  1.67it/s]Results saved to v2\predict
3400 labels saved to v2\predict\labels
 99%|█████████▉| 422/425 [06:17<00:01,  1.67it/s]Results saved to v2\predict
3400 labels saved to v2\predict\labels
100%|█████████▉| 423/425 [06:18<00:01,  1.68it/s]Results saved to v2\predict
3400 labels saved to v2\predict\labels
100%|█████████▉| 424/425 [06:18<00:00,  1.69it/s]Results saved to v2\predict
3400 labels saved to v2\predict\labels
100%|██████████| 425/425 [06:19<00:00,  1.12it/s]


In [12]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):    
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]
    
    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)
    
    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [13]:
infer_txts = glob(f"{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0].split("\\")[1]
    imgage_height, imgage_width = cv2.imread(f"data/test/{base_file_name}.png").shape[:2]        
    with open(infer_txt, "r") as reader:        
        lines = reader.readlines()        
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission['file_name'] = df_submission.file_name.apply(lambda x : x.split("\\")[1])
df_submission.to_csv(f"Submission/YOLO8x_V2_epoch200_Adamw_640_2.csv", index=False)

100%|██████████| 3400/3400 [02:03<00:00, 27.49it/s]


In [14]:
df_submission

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,064442001.png,14,0.815659,1137,180,1427,180,1427,492,1137,492
1,064442001.png,7,0.558712,1137,181,1428,181,1428,498,1137,498
2,064442001.png,7,0.974962,1139,182,1430,182,1430,494,1139,494
3,064442001.png,16,0.900185,1140,182,1432,182,1432,493,1140,493
4,064442001.png,16,0.929843,1138,182,1430,182,1430,495,1138,495
...,...,...,...,...,...,...,...,...,...,...,...
28747,184121675.png,27,0.956841,876,551,1151,551,1151,860,876,860
28748,184121675.png,27,0.973454,877,554,1150,554,1150,862,877,862
28749,184121675.png,27,0.976038,876,552,1151,552,1151,861,876,861
28750,184121675.png,22,0.839533,875,547,1152,547,1152,864,875,864


In [ ]:
df_submission.to_csv(f"Submission/YOLO8_V2_epoch100.csv", index=False)